In [ ]:
%pylab notebook
import matplotlib.pylab as plt
import pandas as pd
import re
import statsmodels.formula.api as smf
pd.set_option('display.max_columns', 360)
pd.set_option('display.max_rows', 360)

In [ ]:
modice_file = '/Users/brodzik/projects/MODICE/data/MASCONS/mascon_areas_by_year.v02.csv'
modice = pd.read_csv(modice_file)
modice.set_index('Year', inplace=True, verify_integrity=True, drop=True)

In [ ]:
seaice_file = '/Users/brodzik/projects/MODICE/data/sii/nsidc0051_year_by_month.csv'
seaice = pd.read_csv(seaice_file)
seaice.set_index('Year', inplace=True, verify_integrity=True, drop=True)
seaice.drop([1999], inplace=True)
seaice

In [ ]:
# Calculate residuals
# Subtract the column mean from every element in both Data Frames
# QUESTION:  should these be normalized to a Z-score by dividing by the stdev??
modice_res = modice - modice.mean()
seaice_res = seaice - seaice.mean()

In [ ]:
seaice_res

In [ ]:
modice_res

In [ ]:
# Get a list of all Arctic ocean areas (these will be grouped together in the graphics)
seaice.columns
p = re.compile(r'01_(.*)')
ocean_region_labels = [p.search(col).group(1) for col in seaice.columns if p.search(col)]
ocean_region_labels

In [ ]:
# Get a list of all MASCON areas (these will be rows in the display graphic)
modice.columns

In [ ]:
p = re.compile(r'(.+_1strike_MODICE_area_km\^2)')
mascon_labels1 = [p.search(col).group(1) for col in modice.columns if p.search(col)]
mascon_labels1

In [ ]:
p = re.compile(r'(.+)_.strike')
mascons1 = [p.search(label).group(1) for label in mascon_labels1]
mascons1

In [ ]:
modice1_res = modice_res[mascon_labels1]
modice1_res

In [ ]:
my_modice_res = modice1_res["NHem_1strike_MODICE_area_km^2"]
my_modice_res

In [ ]:
my_seaice_res = seaice_res["01_total_area_km2"]
my_seaice_res

In [ ]:
def plot_regression(ax, x, y, pthreshold=0.05, following_year=False):
        
    # To use statsmodels, data need to be in a DataFrame
    # Thanks to https://github.com/justmarkham/DAT4/blob/master/notebooks/08_linear_regression.ipynb
    # for pointers to statsmodels
    if following_year:
        print("doing following year")
        data = pd.concat([y, x.shift(periods=-1)], axis=1).reset_index()
    else:
        data = pd.concat([y, x], axis=1).reset_index()
        
    data.columns = ['Year', 'y', 'x']
    data.dropna(inplace=True)

    # Calculate OLS regression
    try:
        lm = smf.ols(formula='y ~ x', data=data).fit()
    except ValueError:
        print("Value error in OLS call, skipping this plot")
        ax.text(0.5, 0.5, "No Data", horizontalalignment='center')
        ax.xaxis.set_ticklabels([])
        ax.yaxis.set_ticklabels([])
        return
    
    x_new = pd.DataFrame({'x': [data.x.min(), data.x.max()]})
    preds = lm.predict(x_new)
    if lm.pvalues.ix['x'] < pthreshold:
        color = 'r'
        style = '-'
    else:
        color = 'k'
        style = ':'
    
    ax.scatter(x=data.x, y=data.y)
    ax.axhline(y=0.0, color='k', linestyle=":")
    ax.axvline(x=0.0, color='k', linestyle=":")
    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    
    ax.plot(x_new, preds, color=color, linestyle=style, linewidth=2)    

In [ ]:
fig, ax = plt.subplots(1, 2)
plot_regression(ax=ax[0], x=my_seaice_res, y=my_modice_res, pthreshold=0.05)
ax[0].set_ylabel('NHem')
ax[0].set_title('Month 01')
plot_regression(ax=ax[1], x=my_seaice_res, y=my_modice_res, following_year=True, pthreshold=0.05)
ax[1].set_ylabel('NHem')
ax[1].set_title('Month 01 of Next year')
#help(ax)

In [ ]:
def mascon_list(area='big'):
    if area == 'big':
        return(['NHem', 'Arctic', 'Greenland'])
    elif area == 'arctic':
        return(['Alaskag',
                'NW_Amer',
                'Baffing',
                'Ellesme',
                'FrnJLnd',
                'Iceland',
                'Nov_Zem',
                'Scandin',
                'Sev_Zem',
                'Sib+Kam',
                'Svalbar'])
    elif area == 'hma':
        return(['Altaigl',
                'HghMtnA',
                'Him+Kar',
                'Nthasia',
                'Pam+Kun',
                'Tianshn',
                'Tib+Qil'])

In [ ]:
def do_full_comparison(seaice_variable='total_area_km2', nstrikes=1, pthreshold=0.05):
    print("Next variable=%s" % seaice_variable)
    areas = ['big', 'arctic', 'hma']
    suptitle_offset = [0.9, 0.96, 0.93]
    for area_idx, area in enumerate(areas):
        print("Next area=%s" % area)
        ncols = 24
        cols = np.arange(ncols) + 1
        mascons = mascon_list(area)
        mascon_labels = ["%s_%dstrike_MODICE_area_km^2" % (mascon, nstrikes) for mascon in mascons]
        nmascons = len(mascon_labels)
        scale = 2
        fig, ax = plt.subplots(nmascons, ncols, figsize=(scale*ncols, scale*nmascons))
        fig.suptitle("Seaice (%s) vs. MODICE(%d strike) mascon (p < %04.2f)" % 
                     (seaice_variable, nstrikes, pthreshold), fontsize=16)
        for row, modice_label in enumerate(mascon_labels):
            for col, month_index in enumerate(cols):
                print("row, col = %d, %d" % (row, col))
                if month_index > 12:
                    month = month_index - 12
                else:
                    month = month_index
                column_label = "%02d_%s" % (month, seaice_variable)

                plot_regression(ax=ax[row, col], 
                                x=seaice_res[column_label],
                                y=modice1_res[modice_label],
                                following_year=month_index > 12,
                                pthreshold=pthreshold)
        
                if col==0:
                    ax[row, col].set_ylabel("%s MODICE" % (mascons[row]))
                if row==0:
                    if month_index > 12:
                        ax[row, col].set_title("next %d seaice" % (month))
                    else:
                        ax[row, col].set_title("%d seaice" % (month))

        fig.subplots_adjust(wspace=0, hspace=0)
        #plt.tight_layout()

        # Adjust the final layout so the main title is set right
        fig.subplots_adjust(top=suptitle_offset[area_idx])
        
        fig.savefig("/Users/brodzik/projects/MODICE/data/MASCONS/MODICE1_%s_vs_%s.p%04.2f.png" %
                    (area, seaice_variable, pthreshold))
    
    plt.close('all')

In [ ]:
test_label = ocean_region_labels[9:]
#for ocean_region_label in ocean_region_labels:
for ocean_region_label in test_label:
    do_full_comparison(seaice_variable=ocean_region_label)

In [ ]:
fig, ax = plt.subplots(1)
ax.text(0.5, 0.5, "this is a test", horizontalalignment='center')

In [ ]:
mascons = mascon_list('big')
mascon_labels = ["%s_1strike_MODICE_area_km^2" % mascon for mascon in mascons]
print(mascons)
print(mascon_labels)

In [ ]:
ocean_region_labels[9:]